# Raster and training data preparation is CSC's Puhti supercomputer

You can get acces to CSC's services from https://research.csc.fi/

### Import neccessary libraries

In [1]:
import csv
import glob
import logging
import os
from tempfile import TemporaryDirectory
import glob

from osgeo import gdal
import joblib
import numpy as np
from osgeo import osr, ogr
from scipy import sparse as sp
#from sklearn import ensemble as ens
#from sklearn.model_selection import cross_val_score
import rasterio as rio
from sklearn.decomposition import PCA

### Helper functions:

In [2]:
def list_rasters(year,statistic):
    " Utility to create file lists of sentinel-1 rasters in Puhti from 01.05 to 30.9 with a given year and statistic"
    # Use glob.glob to list rasters that match the search pattern
    VH_rasters = glob.glob("/appl/data/geo/sentinel/s1/*_grd_{}0[5-9]*_{}_{}*.tif".format(year,statistic, 'VH'))
    # Sort the raster list by acquisition date
    VH_rasters = sorted(VH_rasters)
    VV_rasters = glob.glob("/appl/data/geo/sentinel/s1/*_grd_{}0[5-9]*_{}_{}*.tif".format(year,statistic, 'VV'))
    VV_rasters = sorted(VV_rasters)
    return VH_rasters, VV_rasters

In [3]:
def clip_raster(input_image,year,polarization,statistic,bounds=[248000.000, 6750000.000, 488000.000, 6606000.000]):
    " Utility that clips the  given national wide raster with the bounding box of the Uusimaa region"
    print("processing image {}".format(input_image))
    

    ras_path = os.path.join('/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/{}/{}/{}'.format(year,polarization,statistic), 
                            'UM_{}'.format(os.path.basename(input_image)))
    # Set clipping options
    ras_params = gdal.TranslateOptions(
            noData=0,
            xRes=20,
            yRes=20,
            projWin = bounds
        )
    
    if not os.path.exists(ras_path):
        # Perform the clipping with gdal translate
        gdal.Translate(ras_path, input_image, options=ras_params)
    else:
        print('File already exists')
    return ras_path
    

In [4]:
def loop_rasters(rasters,year,polarization,statistic):
    "Utility that feeds the given raster list to the clipping function"
    out_rasters = []
    for raster in rasters:
        out_raster = clip_raster(raster,year,polarization,statistic)
        out_rasters.append(out_raster)
    return out_rasters

In [5]:
def get_rasters():
    " Utility that first list the nation wide raster, calls the functions above to clip the rasters and returns an array of the clipped rasters"
    mean20_vh, mean20_vv = list_rasters(2020,'mean')
    max20_vh, max20_vv = list_rasters(2020,'max')
    mean21_vh, mean21_vv = list_rasters(2021,'mean')
    max21_vh, max21_vv = list_rasters(2021,'max')
    
    #print(mean20_vv)
    
    um_20_mean_vh = loop_rasters(mean20_vh,2020,'VH','mean')
    um_20_mean_vv = loop_rasters(mean20_vv,2020,'VV','mean')
    um_21_mean_vh = loop_rasters(mean21_vh,2021,'VH','mean')
    um_21_mean_vv = loop_rasters(mean21_vv,2021,'VV','mean')
    
    um_20_max_vh = loop_rasters(max20_vh,2020,'VH','max')
    um_20_max_vv = loop_rasters(max20_vv,2020,'VV','max')
    um_21_max_vh = loop_rasters(max21_vh,2021,'VH','max')
    um_21_max_vv = loop_rasters(max21_vv,2021,'VV','max')
    
    return [um_20_mean_vh,um_20_mean_vv,um_21_mean_vh,um_21_mean_vv,um_20_max_vh,um_20_max_vv,um_21_max_vh,um_21_max_vv]

In [6]:
raster_array = get_rasters()

processing image /appl/data/geo/sentinel/s1/s1m_grd_20200501-20200511_mean_VH_R20m.tif
File already exists
processing image /appl/data/geo/sentinel/s1/s1m_grd_20200511-20200521_mean_VH_R20m.tif
File already exists
processing image /appl/data/geo/sentinel/s1/s1m_grd_20200521-20200531_mean_VH_R20m.tif
File already exists
processing image /appl/data/geo/sentinel/s1/s1m_grd_20200601-20200611_mean_VH_R20m.tif
File already exists
processing image /appl/data/geo/sentinel/s1/s1m_grd_20200611-20200621_mean_VH_R20m.tif
File already exists
processing image /appl/data/geo/sentinel/s1/s1m_grd_20200621-20200701_mean_VH_R20m.tif
File already exists
processing image /appl/data/geo/sentinel/s1/s1m_grd_20200701-20200711_mean_VH_R20m.tif
File already exists
processing image /appl/data/geo/sentinel/s1/s1m_grd_20200721-20200731_mean_VH_R20m.tif
File already exists
processing image /appl/data/geo/sentinel/s1/s1m_grd_20200801-20200811_mean_VH_R20m.tif
File already exists
processing image /appl/data/geo/senti

In [7]:
def harmonize_raster_lists(raster_array):
    "Utility that makes equal length raster lists"
    idx = [0,1,4,5]
    for i in idx:
        raster_array[i].pop(11)
        raster_array[i+2].pop(12)
        raster_array[i+2].pop(7)
    return raster_array

    

In [8]:
for file in raster_array[6]:
    print(os.path.basename(file))

UM_s1m_grd_20210501-20210511_max_VH_R20m.tif
UM_s1m_grd_20210511-20210521_max_VH_R20m.tif
UM_s1m_grd_20210521-20210531_max_VH_R20m.tif
UM_s1m_grd_20210601-20210611_max_VH_R20m.tif
UM_s1m_grd_20210611-20210621_max_VH_R20m.tif
UM_s1m_grd_20210621-20210701_max_VH_R20m.tif
UM_s1m_grd_20210701-20210711_max_VH_R20m.tif
UM_s1m_grd_20210711-20210721_max_VH_R20m.tif
UM_s1m_grd_20210721-20210731_max_VH_R20m.tif
UM_s1m_grd_20210801-20210811_max_VH_R20m.tif
UM_s1m_grd_20210811-20210821_max_VH_R20m.tif
UM_s1m_grd_20210821-20210831_max_VH_R20m.tif
UM_s1m_grd_20210911-20210921_max_VH_R20m.tif
UM_s1m_grd_20210921-20211001_max_VH_R20m.tif


In [9]:
harmonized_raster_array = harmonize_raster_lists(raster_array)

Create folders if they do not exist. Note that you can run command line commands from Jupyter Notebook using !

In [51]:
!mkdir /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/max/
!mkdir /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/mean/VH
!mkdir /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/max/VV
!mkdir /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/max/VH
!mkdir /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/mean/VV

mkdir: cannot create directory ‘/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/max/’: File exists
mkdir: cannot create directory ‘/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/mean/VH’: File exists
mkdir: cannot create directory ‘/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/max/VV’: File exists
mkdir: cannot create directory ‘/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/max/VH’: File exists


In [39]:
def difference_image(image_1, image_2):
    "Calculate a difference image between two input images"
    
    # Open images with rasterio
    ds1 = rio.open(image_1)
    ds2 = rio.open(image_2)
    
    # Read image bands to numpy arrays
    b1 = ds1.read(1)
    b2 = ds2.read(1)
    
    # Convert decibel values into linear 
    b1_linear = 10**(b1/10)
    b2_linear = 10**(b2/10)
    
    # Calculate the ratio image
    ratio_linear = b2_linear/b1_linear
  

    start_date = os.path.basename(image_1).split('_')[3].split('-')[0]
    end_date = os.path.basename(image_2).split('_')[3].split('-')[1]

    statistic = os.path.basename(image_1).split('_')[4]
    polarization = os.path.basename(image_1).split('_')[5]

    ratio_image_name = f'UM_{statistic}_{polarization}_{start_date}-{end_date}_ratio.tif'
    

    ratio_image_path = os.path.join('/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images',statistic,polarization,ratio_image_name)

    # Create an rasterio environment to save the ratio image
    with rio.Env():

        # Copy image profile from the first input image
        profile = ds1.profile

         # Specify a new no-data value
        profile.update(
            # set no data value
            nodata=-3.40282e+38
        )
        # Save the ratio image
        with rio.open(ratio_image_path, 'w', **profile) as dst:
            dst.write(ratio_linear, 1)

    ds1.close()
    ds2.close()

    return ratio_linear


In [56]:
def pca_image(image_1, image_2):
    
    "Transfrom two input images to their principal components using principal component analysis PCA"
    # open the images and read in their band1
    ds1 = rio.open(image_1)
    ds2 = rio.open(image_2)
    b1 = ds1.read(1)
    b2 = ds2.read(1)
    
    # Create image stack of two images
    stack = np.stack((b1,b2))
    # flatten the stack to 1D 
    stack = np.transpose(stack, (1, 2, 0))
    stack = stack.reshape(-1,2)
    
    # Initialize the PCA
    pca = PCA(n_components=2)
    # Perform the PCA
    pca.fit(stack)
    
    print('Relative variance in principal components:',
      pca.explained_variance_ratio_)
    
    # Transfrom the images to their principal components
    predict = pca.transform(stack)
    
    # Reshape the transformed 1D array back to image
    pca_img  = np.reshape(predict, (7200, 12000,2))
    pca_img = np.transpose(pca_img, (2,0,1))
    pca_img = pca_img[0]
    
    start_date = os.path.basename(image_1).split('_')[3].split('-')[0]
    end_date = os.path.basename(image_2).split('_')[3].split('-')[1]

    statistic = os.path.basename(image_1).split('_')[4]
    polarization = os.path.basename(image_1).split('_')[5]

    pca_image_name = f'UM_{statistic}_{polarization}_{start_date}-{end_date}_pca.tif'
    

    pca_image_path = os.path.join('/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images',statistic,polarization,pca_image_name)

    # Create an rasterio environment to save the ratio image
    with rio.Env():

        # Copy image profile from the first input image
        profile = ds1.profile

        # Specify a new no-data value
        profile.update(
            nodata=-3.40282e+38

        )
        # Save the PCA transformed image
        with rio.open(pca_image_path, 'w', **profile) as dst:
            dst.write(pca_img, 1)

    ds1.close()
    ds2.close()

    return None

### Difference images for VH images

In [50]:
for i in range(len(harmonized_raster_array[0])-1):
    difference_image(harmonized_raster_array[0][i], harmonized_raster_array[0][i+1])
    difference_image(harmonized_raster_array[2][i], harmonized_raster_array[2][i+1])
    difference_image(harmonized_raster_array[4][i], harmonized_raster_array[4][i+1])
    difference_image(harmonized_raster_array[6][i], harmonized_raster_array[6][i+1])

### PCA transform for VH images

In [59]:
for i in range(len(raster_array[0])-1):
    pca_image(harmonized_raster_array[4][i], harmonized_raster_array[4][i+1])
    pca_image(harmonized_raster_array[6][i], harmonized_raster_array[6][i+1])
    pca_image(harmonized_raster_array[0][i], harmonized_raster_array[0][i+1])
    pca_image(harmonized_raster_array[2][i], harmonized_raster_array[2][i+1])

Relative variance in principal components: [0.95530364 0.04469636]
Relative variance in principal components: [0.97207163 0.02792837]
Relative variance in principal components: [0.97479466 0.02520534]
Relative variance in principal components: [0.98596408 0.01403592]
Relative variance in principal components: [0.96863217 0.03136783]
Relative variance in principal components: [0.97828215 0.02171785]
Relative variance in principal components: [0.98478878 0.01521122]
Relative variance in principal components: [0.98845217 0.01154783]
Relative variance in principal components: [0.98296774 0.01703226]
Relative variance in principal components: [0.97118845 0.02881155]
Relative variance in principal components: [0.98800466 0.01199534]
Relative variance in principal components: [0.99094259 0.00905741]
Relative variance in principal components: [0.96542885 0.03457115]
Relative variance in principal components: [0.94512561 0.05487439]
Relative variance in principal components: [0.98894593 0.01105

## Virtual rasters

In [40]:
# Create again more folders
!mkdir /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt
!mkdir /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/stacks

mkdir: cannot create directory ‘/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt’: File exists
mkdir: cannot create directory ‘/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/stacks’: File exists


### Virtual raster creation for original VH images

In [34]:
# combine images from year 2020 with images from year 2021
mean_vh_list = raster_array[0] + raster_array[2]
max_vh_list =  raster_array[4] + raster_array[6]

# yearly composites
max_vh_20 = raster_array[4] 
max_vh_21 = raster_array[6] 

# write image lists to file
with open ('/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/mean_vh_list.txt', 'w') as file:
    for image in mean_vh_list:
        file.write(image + "\n")
        
with open ('/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/max_vh_list.txt', 'w') as file:
    for image in max_vh_list:
        file.write(image + "\n")
        
with open ('/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/max_vh_20.txt', 'w') as file:
    for image in max_vh_20:
        file.write(image + "\n")
        
with open ('/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/max_vh_21.txt', 'w') as file:
    for image in max_vh_21:
        file.write(image + "\n")
        
        

# build virtual rasters        
!gdalbuildvrt -separate -overwrite /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/UM_mean_VH.vrt -input_file_list /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/mean_vh_list.txt
!gdalbuildvrt -separate -overwrite /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/UM_max_VH.vrt -input_file_list /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/max_vh_list.txt
!gdalbuildvrt -separate -overwrite /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/UM_max_VH_20.vrt -input_file_list /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/max_vh_20.txt
!gdalbuildvrt -separate -overwrite /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/UM_max_VH_21.vrt -input_file_list /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/max_vh_21.txt

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


In [37]:
! module load qgis

------------------------------------------
QGIS 3.22 and GRASS GIS 7.8.5
https://docs.csc.fi/apps/qgis
https://docs.csc.fi/apps/grass
------------------------------------------

Lmod is automatically replacing "geoconda/3.10.6" with "qgis/3.22".

------------------------------------------
QGIS 3.22 and GRASS GIS 7.8.5
https://docs.csc.fi/apps/qgis
https://docs.csc.fi/apps/grass
------------------------------------------


In [40]:
from osgeo import qgs

ImportError: cannot import name 'qgis' from 'osgeo' (/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/site-packages/GDAL-3.5.0-py3.10-linux-x86_64.egg/osgeo/__init__.py)

In [38]:
from oag
processing.run("native:cellstatistics", {'INPUT':['/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/UM_max_VH_20.vrt'],'STATISTIC':3,'IGNORE_NODATA':True,'REFERENCE_LAYER':'/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/UM_max_VH_20.vrt','OUTPUT_NODATA_VALUE':-9999,'OUTPUT':'/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/UM_max_max_VH_20.tif'})

ModuleNotFoundError: No module named 'qgis'

### Virtual raster creation for ratio images

In [70]:
!gdalbuildvrt -separate -overwrite /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt/UM_mean_VH_ratio.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/mean/VH/*.tif
!gdalbuildvrt -separate -overwrite /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt/UM_max_VH_ratio.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/max/VH/*.tif

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


### Virtual raster creation for PCA transformed images

In [71]:
!gdalbuildvrt -separate -overwrite /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt/UM_max_VH_pca.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/max/VH/*pca.tif
!gdalbuildvrt -separate -overwrite /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt/UM_mean_VH_pca.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/mean/VH/*pca.tif

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


## Raster stacks

In [76]:
!gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt/UM_mean_VH_ratio.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/stacks/UM_mean_VH_ratio.tif
!gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt/UM_max_VH_ratio.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/stacks/UM_max_VH_ratio.tif
!gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt/UM_max_VH_pca.vrt  /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/stacks/UM_max_VH_pca.tif
!gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt/UM_mean_VH_pca.vrt  /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/stacks/UM_mean_VH_pca.tif
!gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/UM_bitemporal_mean.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/UM_bitemporal_mean_VH.tif
!gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/UM_max_VH.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/UM_max_VH.tif
!gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/UM_mean_VH.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/UM_mean_VH.tif


Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.


### Clip rasters to training area

In [75]:
# original time series
! gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/UM_mean_VH.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/training_data/UM_mean_VH_training.tif \
 -projwin 302280.0000000000000000  6718100.0000000000000000  330960.0000000000000000 6701800.0000000000000000
! gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/UM_max_VH.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/training_data/UM_max_VH_training.tif \
 -projwin 302280.0000000000000000  6718100.0000000000000000  330960.0000000000000000 6701800.0000000000000000


# pca images
! gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt/UM_mean_VH_pca.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/training_data/UM_mean_VH_pca_training.tif \
 -projwin 302280.0000000000000000  6718100.0000000000000000  330960.0000000000000000 6701800.0000000000000000
! gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt/UM_max_VH_pca.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/training_data/UM_max_VH_pca_training.tif \
 -projwin 302280.0000000000000000  6718100.0000000000000000  330960.0000000000000000 6701800.0000000000000000
 # ratio images
! gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt/UM_mean_VH_pca.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/training_data/UM_mean_VH_pca_training.tif \
 -projwin 302280.0000000000000000  6718100.0000000000000000  330960.0000000000000000 6701800.0000000000000000
! gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/difference_images/vrt/UM_max_VH_pca.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/training_data/UM_max_VH_pca_training.tif \
 -projwin 302280.0000000000000000  6718100.0000000000000000  330960.0000000000000000 6701800.0000000000000000

# yearly composites
! gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/UM_bitemporal_mean.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/training_data/UM_mean_VH_bitemporal_training.tif \
 -projwin 302280.0000000000000000  6718100.0000000000000000  330960.0000000000000000 6701800.0000000000000000


Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.


### Increased training area

In [8]:
! gdal_translate /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/virtual_rasters/UM_bitemporal_mean.vrt /scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/training_data/UM_mean_VH_bitemporal_training_large.tif \
    -projwin 280430.0 6738772.0 400430.0 6668772.0 
#-a_nodata 255

Input file size is 12000, 7200
0...10...20...30...40...50...60...70...80...90...100 - done.


## Create features and lables for training

In [17]:
base_folder = '/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa'
image_path = '/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/training_data/UM_max_VH_training.tif'
image_path = '/scratch/project_2004990/jutilaee/mtk_kehitys/uusimaa/training_data/UM_mean_VH_bitemporal_training_large.tif'
shape_path = os.path.join(base_folder,'training_data/uusimaa_change_labels_clipped.gpkg')
csv_path = os.path.join(base_folder,'uusimaa_signatures_and_features_max_VH.csv')
csv_path = os.path.join(base_folder,'uusimaa_signatures_and_features_mean_VH_bitemporal.csv')

In [18]:
def get_local_top_left(raster1, raster2):
    """
    NOT MY OWN CODE BUT I DO NOT REMEMBER THE SOURCE
    Gets the top-left corner of raster1 in the array of raster 2.
    Assumes both rasters are in the same projection and units.

    Parameters
    ----------
    raster1 : gdal.Image
        The raster to get the top-left corner of.
    raster2 : gdal.Image
        The raster that raster1's top-left corner is over.

    Returns
    -------
    x_pixel, y_pixel : int
        The indicies of the pixel of top-left corner of raster 1 in raster 2.

    """
    # TODO: Test this
    inner_gt = raster2.GetGeoTransform()
    return point_to_pixel_coordinates(raster1, [inner_gt[0], inner_gt[3]])

In [19]:
def pixel_to_point_coordinates(pixel, GT):
    """
    NOT MY OWN CODE BUT I DO NOT REMEMBER THE SOURCE
    Given a pixel and a geotransformation, returns the picaltion of that pixel's top left corner in the projection
    used by the geotransform.
    NOTE: At present, this takes input in the form of y,x! This is opposite to the output of point_to_pixel_coordinates!

    Parameters
    ----------
    pixel : iterable
        A tuple (y, x) of the coordinates of the pixel
    GT : iterable
        A six-element numpy array containing a geotransform

    Returns
    -------
    Xgeo, Ygeo : number
        The geographic coordinates of the top-left corner of the pixel.

    """
    Xpixel = pixel[1]
    Yline = pixel[0]
    Xgeo = GT[0] + Xpixel * GT[1] + Yline * GT[2]
    Ygeo = GT[3] + Xpixel * GT[4] + Yline * GT[5]
    return Xgeo, Ygeo


In [20]:
def point_to_pixel_coordinates(raster, point, oob_fail=False):
    """
    NOT MY OWN CODE BUT I DO NOT REMEMBER THE SOURCE
    Returns a tuple (x_pixel, y_pixel) in a georaster raster corresponding to the geographic point in a projection.
     Assumes raster is north-up non rotated.

    Parameters
    ----------
    raster : gdal.Image
        A gdal raster object
    point : str, iterable or ogr.Geometry
        One of:
            A well-known text string of a single point
            An iterable of the form (x,y)
            An ogr.Geometry object containing a single point
    Returns
    -------
    A tuple of (x_pixel, y_pixel), containing the indicies of the point in the raster.

    Notes
    -----
    The equation is a rearrangement of the section on affinine geotransform in http://www.gdal.org/gdal_datamodel.html

    """
    if isinstance(point, str):
        point = ogr.CreateGeometryFromWkt(point)
        x_geo = point.GetX()
        y_geo = point.GetY()
    if isinstance(point, list) or isinstance(point, tuple):  # There is a more pythonic way to do this
        x_geo = point[0]
        y_geo = point[1]
    if isinstance(point, ogr.Geometry):
        x_geo = point.GetX()
        y_geo = point.GetY()
    gt = raster.GetGeoTransform()
    x_pixel = int(np.floor((x_geo - floor_to_resolution(gt[0], gt[1]))/gt[1]))
    y_pixel = int(np.floor((y_geo - floor_to_resolution(gt[3], gt[5]*-1))/gt[5]))  # y resolution is -ve
    return x_pixel, y_pixel


In [21]:
def floor_to_resolution(input, resolution):
    """
    NOT MY OWN CODE BUT I DO NOT REMEMBER THE SOURCE
    Returns input rounded DOWN to the nearest multiple of resolution. Used to prevent float errors on pixel boarders.

    Parameters
    ----------
    input : number
        The value to be rounded
    resolution : number
        The resolution

    Returns
    -------
    output : number
        The largest value between input and 0 that is wholly divisible by resolution.

    Notes
    -----
    Uses the following formula: ``input-(input%resolution)``
    If resolution is less than 1, then this assumes that the projection is in decmial degrees and will be rounded to 6dp
    before flooring. However, it is not recommended to use this function in that situation.

    """
    if resolution > 1:
        return input - (input%resolution)
    else:
        log.warning("Low resolution detected, assuming in degrees. Rounding to 6 dp.\
                Probably safer to reproject to meters projection.")
        resolution = resolution * 1000000
        input = input * 1000000
        return (input-(input%resolution))/1000000


In [27]:
def get_training_data(image_path, shape_path, attribute="CODE", shape_projection_id=3067):
    """
    NOT MY OWN CODE BUT I DO NOT REMEMBER THE SOURCE
    
    Given an image and a shapefile with categories, returns training data and features suitable
    for fitting a scikit-learn classifier.

    For full details of how to create an appropriate shapefile, see [here](../index.html#training_data).

    Parameters
    ----------
    image_path : str
        The path to the raster image to extract signatures from
    shape_path : str
        The path to the shapefile containing labelled class polygons
    attribute : str, optional
        The shapefile field containing the class labels. Defaults to "CODE".
    shape_projection_id : int, optional
        The EPSG number of the projection of the shapefile. Defaults to EPSG 4326.

    Returns
    -------
    training_data : array_like
        A numpy array of shape (n_pixels, bands), where n_pixels is the number of pixels covered by the training polygons
    features : array_like
        A 1-d numpy array of length (n_pixels) containing the class labels for the corresponding pixel in training_data

    Notes
    -----
    For performance, this uses scikit's sparse.nonzero() function to get the location of each training data pixel.
    This means that this will ignore any classes with a label of '0'.

    """
    # TODO: WRITE A TEST FOR THIS TOO; if this goes wrong, it'll go wrong
    # quietly and in a way that'll cause the most issues further on down the line
    FILL_VALUE = -9999
    with TemporaryDirectory() as td:
        # Step 1; rasterise shapefile into .tif of class values
        shape_projection = osr.SpatialReference()
        shape_projection.ImportFromEPSG(shape_projection_id)
        image = gdal.Open(image_path)
        image_gt = image.GetGeoTransform()
        x_res, y_res = image_gt[1], image_gt[5]
        ras_path = os.path.join(td, "poly_ras")
        ras_params = gdal.RasterizeOptions(
            noData=0,
            attribute=attribute,
            xRes=x_res,
            yRes=y_res,
            outputType=gdal.GDT_Int16,
            outputSRS=shape_projection,
            #outputBounds=[353359,6705285,377620,6719394]
            #outputBounds = [302280,6701800,330960,6718100]
            outputBounds = [280430, 6668772, 400430, 6738772]
        )
        # This produces a rasterised geotiff that's right, but not perfectly aligned to pixels.
        # This can probably be fixed.
        gdal.Rasterize(ras_path, shape_path, options=ras_params)
        rasterised_shapefile = gdal.Open(ras_path)
        shape_array = rasterised_shapefile.GetVirtualMemArray()
        local_x, local_y = get_local_top_left(image, rasterised_shapefile)
        shape_sparse = sp.coo_matrix(np.asarray(shape_array).squeeze())
        y, x, features = sp.find(shape_sparse)
        training_data = np.empty((len(features), image.RasterCount))
        image_array = image.GetVirtualMemArray()
        image_view = image_array[:,
                    local_y: local_y + rasterised_shapefile.RasterYSize,
                    local_x: local_x + rasterised_shapefile.RasterXSize
                    ]
        for index in range(len(features)):
            training_data[index, :] = image_view[:, y[index], x[index]]
        image_view = None
        image_array = None
        shape_array = None
        rasterised_shapefile = None
        return training_data, features


In [28]:
training_data, features = get_training_data(image_path, shape_path, attribute="CODE", shape_projection_id=3067)

In [29]:
len(features)

37831

In [30]:
sigs = np.vstack((features, training_data.T))

In [31]:
sigs.shape

(3, 37831)

In [32]:
with open(csv_path, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(sigs.T)